<a href="https://colab.research.google.com/github/elbernaderen/final-project/blob/alter/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instalación de python, librerias con mapas y flask en el servidor

In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
# Install flask-ngrok to run flask on colab
!pip install flask-ngrok

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,886 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

# Base de datos

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS claims(name TEXT,
                      address TEXT, 
                      latitude REAL, 
                      longitude REAL, 
                      date TEXT,
                      reason TEXT);''')

conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''CREATE TABLE users (id INTEGER,
                              username TEXT NOT NULL,
                              hash TEXT NOT NULL, 
                              PRIMARY KEY(id))''')
conn.commit()

print("Table created successfully");

conn.close()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 46))



Opened database successfully


OperationalError: ignored

# aplicación

https://stackoverflow.com/questions/52638926/geo-coding-for-intersection-street-using-python from here i have taken the idea for searching latitude and longitude from googlemaps

In [ ]:
import os
from flask_ngrok import run_with_ngrok
from datetime import datetime
from flask import Flask, flash, redirect, render_template, request, session
from geopandas.tools import geocode
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import requests
import sqlite3
import re
from datetime import datetime
from werkzeug.exceptions import default_exceptions, HTTPException, InternalServerError
from werkzeug.security import check_password_hash, generate_password_hash




app = Flask(__name__,template_folder='/content/drive/MyDrive/final/template')
run_with_ngrok(app)
# Ensure templates are auto-reloaded
app.config["SESSION_PERMANENT"] = False
app.config["TEMPLATES_AUTO_RELOAD"] = True
app.config["SESSION_TYPE"] = "filesystem"

lat = -31.4186703
long = -64.1903681

# Ensure responses aren't cached
@app.route("/", methods=["GET", "POST"])
def index():
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        if request.method == "POST":
          name = request.form.get("name")
          direction = request.form.get("direction")
          reason =  request.form.get("reason")
          gps1 = loc(direction)
          gps1[0] = float(gps1[0])
          gps1[1] = float(gps1[1])
          conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], datetime.now(), reason])
          conn.commit()
          return redirect("/claims")
        else:
          m_1 = folium.Map(location = (lat,long), zoom_start = 12)
          cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
          for row in cursor:
            folium.Marker(location=[row[0],row[1]],popup = row[2]).add_to(m_1)         
          conn.close()
          m_1.save('/content/drive/MyDrive/final/template/map.html')
          return render_template("index.html")


@app.route("/claims", methods=["GET", "POST"])
def claims():
    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    m_1 = folium.Map(location = (lat,long), zoom_start = 12)
    cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
    
    for row in cursor:
      folium.Marker(location=[row[0],row[1]],popup = row[2]).add_to(m_1)         
    conn.close()
    m_1.save('/content/drive/MyDrive/final/template/map.html')
    return render_template('claims.html')

@app.route("/register", methods=["GET", "POST"])
def register():
    """Register user"""
    if request.method == "POST":
        rows = conn.execute("SELECT * FROM users WHERE username = ?", request.form.get("username"))
        conn.commit()
        # Ensure username exists
        if not request.form.get("password"):
            return apology("Password is empty", 400)

        if len(request.form.get("password")) < 8:
            return apology("Password must be at least 8 characters ", 400)

        password = request.form.get("password")

        confirmation = request.form.get("confirmation")

        if password != confirmation:
            return apology("passwords are different", 400)

        phash = generate_password_hash(password)
        db.execute("INSERT INTO users (username, hash) VALUES(?, ?)", request.form.get("username"), phash)

        rows = db.execute("SELECT * FROM users WHERE username = ?", request.form.get("username"))

        session["user_id"] = rows[0]["id"]

        # Redirect user to home page
        return redirect("/")
    else:
        return render_template("register.html")
def loc(direction):
    try:
      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina")
      #print(res.content)
      return re.findall(r'll=(.*?)" item', res.text)[0].split(',')
    except IndexError:
      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina")
      #print(res.content)
      rest = re.findall(r'null,null,-(.*?)]', res.text)[0].split(',')
      rest[0] = float(rest[0]) * -1  
      return rest
       
@app.route('/map')
def map():
    return render_template('map.html')
app.run()
  


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d73de4182c42.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Jun/2021 01:04:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:04:50] "GET /final/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2021 01:04:50] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:04:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2021 01:04:52] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:04:52] "GET /final/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2021 01:33:59] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:33:59] "GET /final/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2021 01:34:02] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:34:02] "GET /final/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Jun/2021 01:34:07] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [18/Jun/2021 01:34:07] "GET /final/static/styles.css HTTP/1.1" 404 -
